<a href="https://colab.research.google.com/github/nestorsgarzonc/deep_learning_with_flutter/blob/master/Flower_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os

In [2]:
URL='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

In [3]:
zip_file=tf.keras.utils.get_file(origin=URL, fname='flower_photos.tgz', extract=True, cache_subdir='/content')
base_dir=os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 1s 0us/step


In [4]:
IMAGE_SIZE=224
BATCH_SIZE=64

datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
)

In [5]:
train_generator=datagen.flow_from_directory(
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
    )

validation_generator=datagen.flow_from_directory(
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
    )

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [6]:
print(train_generator.class_indices)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [7]:
 labels='\n'.join(sorted(train_generator.class_indices.keys()))

In [8]:
with open('labels.txt', 'w') as f:
  f.write(labels)

In [9]:
IMG_SHAPE=(IMAGE_SIZE, IMAGE_SIZE, 3)

base_model=tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
    )

9412608/9406464 [==============================] - 0s 0us/step


In [10]:
base_model.trainable=False

In [11]:
model=tf.keras.Sequential(
    [
     base_model,
     tf.keras.layers.Conv2D(32, 3, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.GlobalAveragePooling2D(),
     tf.keras.layers.Dense(5, activation='softmax'),
    ]
)

In [12]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [13]:
epochs=10
history=model.fit(train_generator,
                  epochs=epochs,
                  validation_data=validation_generator)

Epoch 1/10
46/46 [==============================] - 49s 322ms/step - loss: 1.7023 - accuracy: 0.5302 - val_loss: 0.5028 - val_accuracy: 0.8235
Epoch 2/10
46/46 [==============================] - 13s 277ms/step - loss: 0.3864 - accuracy: 0.8723 - val_loss: 0.4273 - val_accuracy: 0.8495
Epoch 3/10
46/46 [==============================] - 13s 276ms/step - loss: 0.2423 - accuracy: 0.9148 - val_loss: 0.4741 - val_accuracy: 0.8591
Epoch 4/10
46/46 [==============================] - 13s 277ms/step - loss: 0.2084 - accuracy: 0.9249 - val_loss: 0.4185 - val_accuracy: 0.8632
Epoch 5/10
46/46 [==============================] - 13s 276ms/step - loss: 0.0991 - accuracy: 0.9745 - val_loss: 0.4397 - val_accuracy: 0.8632
Epoch 6/10
46/46 [==============================] - 13s 278ms/step - loss: 0.0691 - accuracy: 0.9832 - val_loss: 0.4798 - val_accuracy: 0.8605
Epoch 7/10
46/46 [==============================] - 13s 281ms/step - loss: 0.0603 - accuracy: 0.9856 - val_loss: 0.5003 - val_accuracy: 0.8687

In [14]:
saved_model_dir=''
tf.saved_model.save(model, saved_model_dir)

converter=tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model=converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets
